# Get data as protobuf example

This notebook shows how to use the Timeseries API to get data in protobuf format. Normally you would get the `.proto` file and generate stubs for the library, but to not include a lot of generated files in the repo, this uses dynamic message generation based on a self describing protobuf description.

Install the `protobuf` package.

In [ ]:
%pip install protobuf

Setup the protobuffer descriptor. This is not necessary when using stubs generated from `.proto` file.

In [ ]:
import google.protobuf.descriptor_pool
from google.protobuf.descriptor_pb2 import FileDescriptorSet
from google.protobuf.message_factory import MessageFactory
from google.protobuf.json_format import MessageToJson

pool = google.protobuf.descriptor_pool.Default()
factory = MessageFactory()

with open("timeseries.protoset", "rb") as proto_file:
    file_descriptor_set = FileDescriptorSet.FromString(proto_file.read())
    for f in file_descriptor_set.file:
        pool.Add(f)

Import the required libraries. The `azure.identity` package will handle the actual authentication to the API

In [ ]:
from omnia_timeseries.api import TimeseriesAPI, TimeseriesEnvironment
from azure.identity import DeviceCodeCredential
import os

Set up the API with the desired environment

In [ ]:
credentials = DeviceCodeCredential(client_id=os.environ["AZURE_CLIENT_ID"], tenant_id=os.environ["AZURE_TENANT_ID"])
api = TimeseriesAPI(azure_credential=credentials, environment=TimeseriesEnvironment.Beta())

Invoking the API with device code credentials will display a code and a url to enter in a browser window. After this is completed, the execution will proceed. The `accept` parameter is set to `application/protobuf` to receive data in protobuf format.

In [ ]:
timeseries_id = os.environ["OMNIA_TIMESERIES_ID"]
protobuf_data = api.get_latest_datapoint(id=timeseries_id, accept="application/protobuf")
desc = pool.FindMessageTypeByName("CompactDatapointsDto")
CompactDataPointsDto = factory.CreatePrototype(desc)
msg = CompactDataPointsDto.FromString(protobuf_data)
print(MessageToJson(msg))

